In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## FastText with AiHub Data (Phase 2)
- Part 2는 Part 1에서 만든 토큰화&자모 분리 데이터를 바탕으로 FastText 모델을 학습시킵니다.

[2024.04.04]
- [링크](https://museonghwang.github.io/nlp(natural%20language%20processing)/2023/02/10/nlp-kor-fasttext/) 참고하여 자모 분리 및 FastText 학습시도

[2024.04.05]
- Colab 환경에서 Mecab을 이용하여 약 19만개를 모두 토큰화 및 자모 분리 시도 -> ⚠ 끝날 기미가 안 보임
- 클래스 비율에 맞게 데이터 크기를 줄이고(50,00,개) ```바른```으로 토큰화 및 자모 분리 시도
- 크기가 줄어든 데이터로 FastText 학습

###0. 라이브러리 설치 및 불러오기

In [2]:
# 필요한 패키지 다운로드 - 한글 자모 단위 처리
!pip install hgtk

In [3]:
# 필요한 패키지 다운로드 - FastText (gensim과는 다름)
!git clone https://github.com/facebookresearch/fastText.git
%cd fastText
!make
!pip install .

fatal: destination path 'fastText' already exists and is not an empty directory.
/content/fastText
make: Nothing to be done for 'opt'.
Processing /content/fastText
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4239625 sha256=fcf70099750284704dabefe5813aea39fad0884cc2d0a420d2b9ccc91e14e704
  Stored in directory: /tmp/pip-ephem-wheel-cache-5gk8fice/wheels/8b/05/af/3cfae069d904597d44b309c956601b611bdf8967bcbe968903
Successfully built fasttext
  Attempting uninstall: fasttext
    Found existing installation: fasttext 0.9.2
    Uninstalling fasttext-0.9.2:
      Successfully uninstalled fasttext-0.9.2


In [4]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from tqdm import tqdm
import pickle
import hgtk
import fasttext

### 1. 토큰화&자모 분리 완료 데이터 불러오기

In [5]:
with open("/content/drive/MyDrive/Projects/random_p2_tokenized_data.pkl","rb") as f:
    tokenized_data = pickle.load(f)

In [6]:
# 확인
tokenized_data[0][:10]

['ㅇㅏㅍ',
 'ㅇㅡ-ㄹㅗ-',
 'ㅅㅏ-ㅇㅓㅂㅈㅏ-',
 'ㄱㅏ-',
 'ㅅㅡ-ㅁㅏ-ㅌㅡ-ㅍㅗㄴ',
 'ㅇㅣ-ㄴㅏ-',
 'USB',
 'ㄷㅡㅇ',
 'ㅇㅔ-',
 'ㅌㅏ-ㅇㅣㄴ']

### 2. 필요한 함수 정의

In [7]:
def word_to_jamo(token):
    def to_special_token(jamo):
        if not jamo:
            return '-'
        else:
            return jamo

    decomposed_token = ''
    for char in token:
        try:
            # char(음절)을 초성, 중성, 종성으로 분리
            cho, jung, jong = hgtk.letter.decompose(char)

            # 자모가 빈 문자일 경우 특수문자 -로 대체
            cho = to_special_token(cho)
            jung = to_special_token(jung)
            jong = to_special_token(jong)
            decomposed_token = decomposed_token + cho + jung + jong

        # 만약 char(음절)이 한글이 아닐 경우 자모를 나누지 않고 추가
        except Exception as exception:
            if type(exception).__name__ == 'NotHangulException':
                decomposed_token += char

    # 단어 토큰의 자모 단위 분리 결과를 추가
    return decomposed_token

In [8]:
def jamo_to_word(jamo_sequence):
    tokenized_jamo = []
    index = 0

    # 1. 초기 입력
    # jamo_sequence = 'ㄴㅏㅁㄷㅗㅇㅅㅐㅇ'

    while index < len(jamo_sequence):
        # 문자가 한글(정상적인 자모)이 아닐 경우
        if not hgtk.checker.is_hangul(jamo_sequence[index]):
            tokenized_jamo.append(jamo_sequence[index])
            index = index + 1

        # 문자가 정상적인 자모라면 초성, 중성, 종성을 하나의 토큰으로 간주.
        else:
            tokenized_jamo.append(jamo_sequence[index:index + 3])
            index = index + 3

    # 2. 자모 단위 토큰화 완료
    # tokenized_jamo : ['ㄴㅏㅁ', 'ㄷㅗㅇ', 'ㅅㅐㅇ']

    word = ''
    try:
        for jamo in tokenized_jamo:

            # 초성, 중성, 종성의 묶음으로 추정되는 경우
            if len(jamo) == 3:
                if jamo[2] == "-":
                    # 종성이 존재하지 않는 경우
                    word = word + hgtk.letter.compose(jamo[0], jamo[1])
                else:
                    # 종성이 존재하는 경우
                    word = word + hgtk.letter.compose(jamo[0], jamo[1], jamo[2])
            # 한글이 아닌 경우
            else:
                word = word + jamo

    # 복원 중(hgtk.letter.compose) 에러 발생 시 초기 입력 리턴.
    # 복원이 불가능한 경우 예시) 'ㄴ!ㅁㄷㅗㅇㅅㅐㅇ'
    except Exception as exception:
        if type(exception).__name__ == 'NotHangulException':
            return jamo_sequence

    return word

In [ ]:
# 지금은 적용할 필요가 없어서 넘어가겠습니다.
result = []
for w in tokenized_data[0]:
    result.append(jamo_to_word(w))

### 3. FastText

In [9]:
# 먼저 훈련데이터를 txt 파일로 만들겠습니다.
with open('tokenized_data.txt', 'w', encoding='utf-8-sig') as out:
    for line in tqdm(tokenized_data, unit=' line'):
        out.write(' '.join(line) + '\n')

100%|██████████| 49999/49999 [00:03<00:00, 14512.12 line/s]


In [10]:
# skip-gram과 cbow 중에서 cbow를 시도해보겠습니다.
model = fasttext.train_unsupervised('tokenized_data.txt', model='cbow')
model.save_model("fasttext.bin") # 모델 저장
model = fasttext.load_model("fasttext.bin") # 모델 로드

- 이렇게 모델 학습이 완료되었습니다. 35분 정도 걸렸습니다. 이제 임베딩 벡터를 확인해보겠습니다.
- 이때 자모 단위로 학습했기 때문에 모델의 입력도 자모 단위로 입력해야 합니다.

In [11]:
# 임베딩 결과 확인
model[word_to_jamo('영업')]

array([-1.86826146e+00,  1.21995342e+00, -1.94796753e+00,  3.98976952e-01,
        1.75177324e+00,  1.77325368e-01,  1.80364168e+00, -1.39277279e-01,
       -9.20918941e-01, -1.94111896e+00,  2.20001578e+00,  1.22023857e+00,
        2.80741811e+00,  1.69143713e+00,  2.51078629e+00, -9.93556499e-01,
        2.62929749e+00, -1.20448089e+00, -8.07328448e-02, -8.44077885e-01,
        5.13169527e-01,  2.64756799e-01, -1.84092951e+00,  2.84274578e+00,
       -2.00315523e+00, -1.34603095e+00,  1.54997632e-01, -1.71831012e+00,
        2.52326012e+00,  2.15441748e-01,  8.77778113e-01,  1.85453549e-01,
       -1.86104262e+00,  1.87952912e+00, -1.19333029e+00, -1.03605509e+00,
        2.59505582e+00, -2.16041461e-01, -1.01596808e+00, -1.17059875e+00,
        3.25354934e+00,  2.33785820e+00, -2.44341395e-03, -6.69884562e-01,
       -1.18245673e+00,  1.80434382e+00, -5.54258525e-01, -1.27541590e+00,
        9.71631885e-01, -5.79150736e-01,  1.46131158e+00, -5.92421532e-01,
       -8.87582779e-01,  

In [12]:
# 영업과 가장 가까운 단어 10개
model.get_nearest_neighbors(word_to_jamo('영업'), k=10)

[(0.8996255993843079, 'ㅅㅜㄴㅇㅕㅇㅇㅓㅂ'),
 (0.8723154664039612, 'ㅇㅕㅇㅇㅓㅂㅇㅣㄱ'),
 (0.8520394563674927, 'ㅇㅕㅇㅇㅓㅂㅁㅏㅇ'),
 (0.8319339156150818, 'ㅇㅕㅇㅇㅓㅂㅈㅣㄱ'),
 (0.8309779763221741, 'ㅇㅕㅇㅇㅓㅂㅇㅣㄹ'),
 (0.8071931600570679, 'ㅇㅕㅇㅇㅓㅂㅈㅜ-'),
 (0.7987553477287292, 'ㅇㅕㅇㅇㅓㅂㄹㅕㄱ'),
 (0.7913650274276733, 'ㅇㅕㅇㅇㅓㅂㅂㅜ-'),
 (0.7566759586334229, 'ㅇㅕㅇㅇㅓㅂㅈㅓㅁ'),
 (0.7537907958030701, 'ㅇㅕㅇㅇㅓㅂㅅㅗ-')]

In [13]:
# 결과를 보기 좋게 다시 단어로 만드는 함수
def transform(word_sequence):
    return [(jamo_to_word(word), similarity) for (similarity, word) in word_sequence]

In [14]:
transform(model.get_nearest_neighbors(word_to_jamo('영업'), k=10))

[('순영업', 0.8996255993843079),
 ('영업익', 0.8723154664039612),
 ('영업망', 0.8520394563674927),
 ('영업직', 0.8319339156150818),
 ('영업일', 0.8309779763221741),
 ('영업주', 0.8071931600570679),
 ('영업력', 0.7987553477287292),
 ('영업부', 0.7913650274276733),
 ('영업점', 0.7566759586334229),
 ('영업소', 0.7537907958030701)]

- 영업이라는 단어를 포함하는 경우가 대부분입니다.

In [15]:
# '제품'라는 단어로도 시도
transform(model.get_nearest_neighbors(word_to_jamo('제품'), k=10))

[('수제품', 0.9000310897827148),
 ('완제품', 0.8986680507659912),
 ('유제품', 0.8916977643966675),
 ('복제품', 0.8841310143470764),
 ('제품명', 0.874809205532074),
 ('시제품', 0.8734177350997925),
 ('제품군', 0.861941397190094),
 ('제품력', 0.858100950717926),
 ('가전제품', 0.8364369869232178),
 ('규격품', 0.7852669954299927)]

- 제품이라는 단어를 포함하는 경우가 대부분입니다.

In [16]:
# '경제'라는 단어로도 시도
transform(model.get_nearest_neighbors(word_to_jamo('경제'), k=10))

[('경제통', 0.9000993371009827),
 ('경제가', 0.8897873759269714),
 ('경제를', 0.8887243270874023),
 ('경제적', 0.8816461563110352),
 ('경제사', 0.8713679313659668),
 ('경제국', 0.8613507747650146),
 ('경제지', 0.8577458262443542),
 ('경제난', 0.8505710363388062),
 ('경제부', 0.8349419832229614),
 ('경제과', 0.8226110339164734)]

- 경제라는 단어를 포함하는 경우가 대부분입니다.
- 이런 경우는 대부분 토큰화 과정에서 접미사로 분리되지 않는 하나의 단어이기 때문에 그럴 것입니다.
- 결과는 상당히 좋은 것 같습니다.

In [18]:
# '뭐야'라는 단어로도 시도(이상한 단어의 결과는?)
transform(model.get_nearest_neighbors(word_to_jamo('뭐야'), k=10))

[('되야', 0.8090258836746216),
 ('려야', 0.7790184020996094),
 ('어야지', 0.7470511198043823),
 ('어야', 0.7273123860359192),
 ('그제서야', 0.7126782536506653),
 ('그거', 0.7099749445915222),
 ('이야', 0.7076672911643982),
 ('어쨌든', 0.6965583562850952),
 ('뭐', 0.6956238150596619),
 ('아야지', 0.6948559284210205)]

- 되야라는 결과가 나온 것은 상당히 재밌는것 같습니다.

In [19]:
# '이거'라는 단어로도 시도
transform(model.get_nearest_neighbors(word_to_jamo('이거'), k=10))

[('그거', 0.8890333771705627),
 ('이것', 0.8240786790847778),
 ('이것저것', 0.8236950635910034),
 ('그러니까', 0.7796077728271484),
 ('라거', 0.7678987383842468),
 ('이러', 0.7427816390991211),
 ('거', 0.7314772605895996),
 ('그러면', 0.7308693528175354),
 ('그런데', 0.729199230670929),
 ('더라니까', 0.7267804741859436)]

- 이거의 유사한 단어는 상당히 잘 나왔습니다.

In [21]:
# '돈'이라는 단어로도 시도
transform(model.get_nearest_neighbors(word_to_jamo('돈'), k=20))

[('뒷돈', 0.8545513153076172),
 ('돈돈', 0.838449239730835),
 ('돈쭐', 0.8188746571540833),
 ('쌈짓돈', 0.809073269367218),
 ('종잣돈', 0.7969065308570862),
 ('큰돈', 0.7603912949562073),
 ('돈줄', 0.7590497732162476),
 ('회삿돈', 0.7489598989486694),
 ('떼돈', 0.7423655390739441),
 ('뭉칫돈', 0.7418103218078613),
 ('목돈', 0.7406257390975952),
 ('나랏돈', 0.7313678860664368),
 ('용돈', 0.7221038937568665),
 ('세뱃돈', 0.7195857167243958),
 ('푼돈', 0.7183853983879089),
 ('여윳돈', 0.7043404579162598),
 ('한돈', 0.7013489603996277),
 ('거금', 0.700924813747406),
 ('잔돈', 0.6924550533294678),
 ('웃돈', 0.6880329847335815)]

In [22]:
# '삼성'이라는 단어로도 시도
transform(model.get_nearest_neighbors(word_to_jamo('삼성'), k=10))

[('삼성SDS', 0.9135909080505371),
 ('삼성SDI', 0.9012464880943298),
 ('삼성홀', 0.8562483191490173),
 ('삼성덱스', 0.8453385829925537),
 ('삼성가', 0.8334404230117798),
 ('삼성에스원', 0.8255721926689148),
 ('삼성역', 0.8245302438735962),
 ('뉴삼성', 0.8222330808639526),
 ('삼성그룹', 0.8053971529006958),
 ('삼성닷컴', 0.8044605255126953)]

- 삼성의 의미를 정확하게 파악하고 있는 것 같습니다.

In [23]:
# 'LG'이라는 단어로도 시도 (영어는?)
transform(model.get_nearest_neighbors(word_to_jamo('LG'), k=10))

[('LGU', 0.9459348917007446),
 ('LGD', 0.9422498345375061),
 ('LGBT', 0.9007177352905273),
 ('LGBTQ', 0.839914858341217),
 ('LG엔솔', 0.7915444374084473),
 ('LG이노텍', 0.7283596992492676),
 ('SK', 0.6889216899871826),
 ('SKT', 0.6878361701965332),
 ('SKB', 0.6848371028900146),
 ('DS', 0.6800515055656433)]

- LG와 유사한 단어가 LGU, LGD입니다. 각각 유플러스, 디스플레이 같습니다.

In [24]:
# '취준생'이라는 단어로도 시도
transform(model.get_nearest_neighbors(word_to_jamo('취준생'), k=10))

[('취준', 0.7517503499984741),
 ('숙명여고', 0.6552878022193909),
 ('취중', 0.6549308896064758),
 ('현생', 0.6450059413909912),
 ('갓생', 0.6357482075691223),
 ('청취자', 0.6291627287864685),
 ('영생', 0.6239936947822571),
 ('전생', 0.6236934661865234),
 ('취직', 0.6210567355155945),
 ('여고', 0.6192942261695862)]

In [25]:
# 'ㄹㅇ'이라는 단어로도 시도
transform(model.get_nearest_neighbors(word_to_jamo('ㄹㅇ'), k=10))

[('ㄹ--', 0.8847793936729431),
 ('ㄹ--ㄹㅗ-', 0.7879843711853027),
 ('ㄹ--ㄲㅔ-', 0.7500530481338501),
 ('ㄹ--ㄴㅡㄴㅈㅣ-', 0.737320065498352),
 ('ㄹ--ㄹㅏ-', 0.7352838516235352),
 ('ㄹ--ㄹㅐ-', 0.7019034028053284),
 ('ㄹ--ㄹㅕ-ㅁㅕㄴ', 0.6997913718223572),
 ('ㄹ--ㅈㅣ-', 0.6932205557823181),
 ('ㄹ--ㅈㅣ-ㄹㅏ-ㄷㅗ-', 0.6876563429832458),
 ('게끔', 0.6822352409362793)]

- 이런 단어는 전혀 못 맞추네요. 데이터 전처리할 때 단어가 아닌 경우를 제거해서 아예 학습되지 않게 하는게 좋을 것 같습니다.

In [26]:
# '사과'라는 단어로도 시도 (한 단어에 여러 뜻이 있다면?)
transform(model.get_nearest_neighbors(word_to_jamo('사과'), k=10))

[('사과문', 0.8473402857780457),
 ('사과하', 0.7733989953994751),
 ('사과나무', 0.747269868850708),
 ('다과', 0.7471315264701843),
 ('타과', 0.7384236454963684),
 ('의사과', 0.7354158163070679),
 ('수사과', 0.6987043619155884),
 ('형사과', 0.6810932755470276),
 ('공개사과', 0.6581310629844666),
 ('사죄', 0.6382438540458679)]

- 먹는 사과와 사과하다에서 사과가 비슷한 유사도를 가질 것이라고 생각했는데 아니네요.

In [27]:
transform(model.get_nearest_neighbors(word_to_jamo('코딩'), k=10))

[('인코딩', 0.8606064915657043),
 ('레코딩', 0.813941240310669),
 ('초딩', 0.7815424799919128),
 ('로딩', 0.7692169547080994),
 ('딩', 0.76865553855896),
 ('키딩', 0.7457128763198853),
 ('워딩', 0.7448025345802307),
 ('튜닝', 0.7425268292427063),
 ('코디스', 0.7367035746574402),
 ('코칭', 0.7263870239257812)]

In [28]:
transform(model.get_nearest_neighbors(word_to_jamo('파이썬'), k=10))

[('파이빅스', 0.8797593116760254),
 ('파이퍼', 0.8584555983543396),
 ('파이트', 0.8548797965049744),
 ('파파이스', 0.8445871472358704),
 ('엘파이스', 0.8410310745239258),
 ('파이버', 0.8409584164619446),
 ('파이어', 0.8364558815956116),
 ('파이팅', 0.8315353393554688),
 ('파이선', 0.8306028842926025),
 ('파이어킴', 0.8208320736885071)]

In [29]:
transform(model.get_nearest_neighbors(word_to_jamo('컴퓨터'), k=10))

[('한글과컴퓨터', 0.8677334785461426),
 ('삼보컴퓨터', 0.8544966578483582),
 ('슈퍼컴퓨터', 0.8528906106948853),
 ('컴퓨팅', 0.8124067783355713),
 ('튜터', 0.7814207673072815),
 ('에지컴퓨팅', 0.7419509291648865),
 ('학습터', 0.7015081644058228),
 ('슈퍼컴퓨팅', 0.6746425628662109),
 ('소프트웨어', 0.6730035543441772),
 ('스플리터', 0.6718708276748657)]

In [30]:
# 5만개로 학습한 FastText 모델 저장
model.save_model("AiHub_Part2_Total_Data_FastText.bin") # 모델 저장

# 모델 로드할 때는...
# model = fasttext.load_model("/content/drive/MyDrive/Projects/AiHub_Part2_Total_Data_FastText.bin")

In [33]:
# model = fasttext.load_model("/content/drive/MyDrive/Projects/AiHub_Part2_Total_Data_FastText.bin")

In [34]:
# transform(model.get_nearest_neighbors(word_to_jamo('영업'), k=10))